# Test some stare mode data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lightkurve as lk
import astropy.units as u
from astropy.io import fits
from glob import glob
import ombre as om
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
name = 'WASP-18'

In [4]:
fnames = np.asarray(glob(f'/Volumes/Nibelheim/hst/data/{name}/*.fits'))
#obs = om.Observation.from_files(fnames)

# Stare Mode Data

In [5]:
propid = []
times = []
for fname in fnames:
    if not os.path.isfile(fname): 
        continue        
    with fits.open(fname, lazy_load_hdus=True) as hdu:
        propid.append(hdu[0].header['PROPOSID'])
        times.append(hdu[0].header['EXPSTART'])

In [6]:
propid = np.asarray(propid)
times = np.asarray(times)
s = np.argsort(times)
fnames, propid, times = fnames[s], propid[s], times[s]

In [10]:
fnames = fnames[np.where(propid == 12181)]

In [11]:
obs = om.Observation.from_files(fnames, force=True)

/Users/ch/HST/projects/ombre/src/ombre/visit.py:227: RuntimeWarning: divide by zero encountered in true_divide
  w[self.err / self.sci > 0.1] = 1e10
/Users/ch/HST/projects/ombre/src/ombre/visit.py:227: RuntimeWarning: divide by zero encountered in true_divide
  w[self.err / self.sci > 0.1] = 1e10


In [ ]:
clc = lk.search_lightcurve(name, author='SPOC').download_all().stitch().remove_nans()
clc.flux = clc.flux.value * u.electron/u.s
clc.flux_err = clc.flux_err.value * u.electron/u.s
k = np.abs(((clc.time.value - (obs.t0 - 2457000 + obs.period/2))/obs.period) % 1 - 0.5) < 0.1
clc[k].fold(obs.period, obs.t0 - 2457000).scatter()

In [ ]:
obs.fit_transit(x_suppl=clc.time.value + 2457000,
                y_suppl=clc.flux.value/np.median(clc.flux.value),
                yerr_suppl=clc.flux_err.value/np.median(clc.flux.value),
                exptime_suppl=np.median(np.diff(clc.time.value)), fit_t0=True, fit_period=True, fit_inc=True)

In [ ]:
obs.plot()

In [ ]:
obs.fit_model()

In [ ]:
om.Spectra([visit.transmission_spectrum for visit in obs]).flatten().bin(bins=np.linspace(0.8, 1.7, 200)).plot()

In [ ]:
spec = obs.stellar_spectra
spec.plot()
spec.hdulist.writeto(f'results/{name}_stellar_spectrum.fits', overwrite=True)

In [ ]:
from astropy.io import fits
try:
    hdulist = om.Spectra([visit.transmission_spectrum for visit in obs if visit.transmission_spectrum.spec.sum() != 0]).hdulist
    hdulist.writeto(f'results/{name}_transmission.fits', overwrite=True)
except:
    pass
try:    
    hdulist = om.Spectra([visit.emission_spectrum for visit in obs if visit.emission_spectrum.spec.sum() != 0]).hdulist
    hdulist.writeto(f'results/{name}_emission.fits', overwrite=True)
except:
    pass

In [ ]:
for visit in obs:
    visit.diagnose();
    plt.show();